In [ ]:
"""This notebook uses the processed files for CAT indices to calculate thresholds for moderate or greater MOG turbulence.
First, we select a mid-lat region for defining MOG.
Then we calculate the threshold or thresholds using quantiles.
Once we have defined a threshold, we can use that value to calculate the frequency of exceeding that threshold.
Frequency above thresholds can be used to evaluate time series and trends over time,
as well as spatial distributions of the strongest areas of turbulence
"""

In [1]:
import xarray as xr
import glob
import intake
import numpy as np

# import warnings
# warnings.filterwarnings('ignore', category=UserWarning)

In [2]:
from plotting_maps.acs_plotting_maps import plot_acs_hazard_multi, plot_acs_hazard, plot_data, cmap_dict, regions_dict
from matplotlib import colors, cm
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt

from dask.diagnostics import ProgressBar
# ProgressBar().register()

import dask
from dask.distributed import Client
client = Client(threads_per_worker=2, n_workers=8)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: /proxy/8787/status,
Dashboard: /proxy/8787/status,Workers: 8
Total threads: 16,Total memory: 125.19 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:46375,Workers: 8
Dashboard: /proxy/8787/status,Total threads: 16
Started: Just now,Total memory: 125.19 GiB
Comm: tcp://127.0.0.1:40171,Total threads: 2
Dashboard: /proxy/39545/status,Memory: 15.65 GiB
Nanny: tcp://127.0.0.1:34901,


2025-09-25 16:54:42,443 - distributed.scheduler - WARNING - Removing worker 'tcp://127.0.0.1:43041' caused the cluster to lose already computed task(s), which will be recomputed elsewhere: {('mean_combine-partial-ac2df34504ef575328fb2d425a8e0359', 0, 0, 0), ('mean_chunk-51fd48ed334d0536e0f1ba84b42c00eb', 0, 1, 6), ('getitem-0627b2b503bf250bbc2008490209f132', 0, 0, 6), 'original-open_dataset-windspeed-f5d560f680af30e0131aecdbb7645845', 'original-open_dataset-windspeed-4f2543bbab7f528f20680e95d45cd5c6', ('mean_combine-partial-4896f385fd5896fb3d2f97a0520924d2', 1, 0, 3), ('mean_combine-partial-ac2df34504ef575328fb2d425a8e0359', 1, 0, 0), ('mean_combine-partial-4896f385fd5896fb3d2f97a0520924d2', 0, 0, 2), ('mean_combine-partial-4896f385fd5896fb3d2f97a0520924d2', 0, 1, 2), ('mean_combine-partial-4896f385fd5896fb3d2f97a0520924d2', 1, 0, 2), ('mean_chunk-51fd48ed334d0536e0f1ba84b42c00eb', 1, 0, 6), ('mean_combine-partial-4896f385fd5896fb3d2f97a0520924d2', 1, 1, 2), ('mean_chunk-51fd48ed334d05

In [32]:
mid_lat_slice = slice(-50,-25)

In [3]:
cat_name = "barpa"
col = intake.open_esm_datastore(f"/g/data/lp01/collections/py3.9_dev/nci-{cat_name}.json")

# Edit this cell
var_list = ["ua200", "ua250", "ua300", "va200", "va250", "va300", "ta200", "ta250", "ta300", "zg200", "zg250", "zg300"]
table_id = "6hr"
scenarios = ["historical","ssp126", "ssp370", "ssp585", "evaluation"]

# change this query to select a subset of the data you are interested in
query = dict(variable_id = var_list[0],
             table_id = table_id,
             experiment_id = scenarios,
            )

cat = col.search(**query)
cat.unique()

activity_id                                                    [BARPA-R]
institution_id                                                     [BOM]
version                                                      [v20231001]
variable_id                                                      [ua200]
table_id                                                           [6hr]
source_id              [ACCESS-CM2, ACCESS-ESM1-5, CESM2, CMCC-ESM2, ...
experiment_id           [historical, ssp126, ssp370, ssp585, evaluation]
member_id                      [r4i1p1f1, r6i1p1f1, r11i1p1f1, r1i1p1f1]
grid_label                                                      [AUS-15]
time_range             [196001-196012, 196101-196112, 196201-196212, ...
path                   [/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM...
derived_variable_id                                                   []
dtype: object

In [4]:
cat_df_max = cat.df.groupby(["variable_id", "experiment_id", "source_id", "member_id"]).max().reset_index()
cat_df_max["index"] = cat_df_max.experiment_id + "_" + cat_df_max.source_id + "_" + cat_df_max.member_id
cat_df_max = cat_df_max.set_index("index")
cat_df_max

,variable_id,experiment_id,source_id,member_id,activity_id,institution_id,version,table_id,grid_label,time_range,path
index,,,,,,,,,,,
evaluation_ERA5_r1i1p1f1,ua200,evaluation,ERA5,r1i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,202001-202012,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...
historical_ACCESS-CM2_r4i1p1f1,ua200,historical,ACCESS-CM2,r4i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,201401-201412,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...
historical_ACCESS-ESM1-5_r6i1p1f1,ua200,historical,ACCESS-ESM1-5,r6i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,201401-201412,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...
historical_CESM2_r11i1p1f1,ua200,historical,CESM2,r11i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,201401-201412,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...
historical_CMCC-ESM2_r1i1p1f1,ua200,historical,CMCC-ESM2,r1i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,201401-201412,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...
historical_EC-Earth3_r1i1p1f1,ua200,historical,EC-Earth3,r1i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,201401-201412,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...
historical_MPI-ESM1-2-HR_r1i1p1f1,ua200,historical,MPI-ESM1-2-HR,r1i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,201401-201412,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...
historical_NorESM2-MM_r1i1p1f1,ua200,historical,NorESM2-MM,r1i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,201401-201412,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...
ssp126_ACCESS-CM2_r4i1p1f1,ua200,ssp126,ACCESS-CM2,r4i1p1f1,BARPA-R,BOM,v20231001,6hr,AUS-15,210001-210012,/g/data/py18/BARPA/output/CMIP6/DD/AUS-15/BOM/...


In [5]:
# indices for evaluation, historical and future groups. These will share time ranges
i_evaluation = cat_df_max.loc[cat_df_max["experiment_id"].isin(["evaluation"])].index
i_historical = cat_df_max.loc[cat_df_max["experiment_id"].isin(["historical"])].index
i_future = cat_df_max.loc[cat_df_max["experiment_id"].isin(["ssp126", "ssp370", "ssp585"])].index

In [6]:
for run in i_future:
    experiment_id, source_id, member_id = run.split("_")
    print(f"{experiment_id}, {source_id}, {member_id}")

ssp126, ACCESS-CM2, r4i1p1f1
ssp126, ACCESS-ESM1-5, r6i1p1f1
ssp126, CESM2, r11i1p1f1
ssp126, CMCC-ESM2, r1i1p1f1
ssp126, EC-Earth3, r1i1p1f1
ssp126, MPI-ESM1-2-HR, r1i1p1f1
ssp126, NorESM2-MM, r1i1p1f1
ssp370, ACCESS-CM2, r4i1p1f1
ssp370, ACCESS-ESM1-5, r6i1p1f1
ssp370, CESM2, r11i1p1f1
ssp370, CMCC-ESM2, r1i1p1f1
ssp370, EC-Earth3, r1i1p1f1
ssp370, MPI-ESM1-2-HR, r1i1p1f1
ssp370, NorESM2-MM, r1i1p1f1
ssp585, ACCESS-CM2, r4i1p1f1
ssp585, EC-Earth3, r1i1p1f1


In [8]:
list_evaluation = ['evaluation_BARRA-R_r1i1p1f1', 'evaluation_ERA5_r1i1p1f1',]

In [ ]:
turbulence_index = "windspeed"
for run in ['evaluation_ERA5_r1i1p1f1']:
    experiment_id, source_id, member_id = run.split("_")
    print(run)
    time_range = np.arange(1990,2009+1)
    
    filelist = [f"/scratch/v46/gt3409/TMP_{turbulence_index}/TMP_{turbulence_index}_AUS-15_{source_id}_{experiment_id}_{member_id}_BOM_BARPA-R_v1-r1_6hr_{year}.nc" 
     for year in time_range]
    
    def _preprocess(ds, q=[0.95, 0.99, 0.999],):
        return ds.sel(lat=mid_lat_slice,).quantile(q, dim=["time", "lat", "lon"])

    ds = xr.open_mfdataset(filelist, use_cftime=True, preprocess=_preprocess, combine="nested", concat_dim="time")
    ds = ds.compute()
    
ds.mean("time")["windspeed"].values    

In [33]:
turbulence_index = "windspeed"
for run in ['evaluation_BARRA-R_r1i1p1f1']:
    experiment_id, source_id, member_id = run.split("_")
    print(run)
    time_range = np.arange(1990,2009+1)
    
    filelist = [f"/scratch/v46/gt3409/TMP_{turbulence_index}/TMP_{turbulence_index}_AUS-15_{source_id}_{experiment_id}_{member_id}_BOM_BARPA-R_v1-r1_6hr_{year}.nc" 
     for year in time_range]
    
    def _preprocess(ds, q=[0.95, 0.99, 0.999],):
        return ds.sel(lat=mid_lat_slice,).quantile(q, dim=["time", "lat", "lon"])

    ds = xr.open_mfdataset(filelist, use_cftime=True, preprocess=_preprocess, combine="nested", concat_dim="time")
    ds = ds.compute()

evaluation_BARRA-R_r1i1p1f1


In [37]:
ds

<xarray.Dataset> Size: 40GB
Dimensions:    (time: 29220, lat: 436, lon: 777)
Coordinates:
    pressure   int64 8B 250
  * time       (time) datetime64[ns] 234kB 1990-01-01 ... 2009-12-31T18:00:00
  * lon        (lon) float64 6kB 88.04 88.19 88.34 88.5 ... 207.6 207.8 207.9
  * lat        (lat) float64 3kB -53.58 -53.42 -53.27 ... 13.32 13.48 13.63
Data variables:
    windspeed  (time, lat, lon) float32 40GB dask.array<chunksize=(1460, 436, 777), meta=np.ndarray>
Attributes:
    regrid_method:  bilinear

In [23]:
# from BARRA-R
# p95, p99, p999 = [62.82748914, 75.75302181, 88.13204733]
p95, p99, p999 = ds.mean("time")["windspeed"].values
p95, p99, p999


(62.84133398056025, 75.76247397613517, 88.13811403617862)

In [11]:
# from era5 (-60,-30)
# (59.650381757029564, 73.46444525842983, 86.41072482237834)
# p95, p99, p999 = ds.mean("time")["windspeed"].values


In [12]:
delayed_results = {}

In [25]:
%%time
turbulence_index = "windspeed"
for run in list_evaluation:
    experiment_id, source_id, member_id = run.split("_")
    name = f"{experiment_id}_{source_id}_{member_id}"
    print(name)
    time_range = np.arange(1990,2014+1)
    # time_range = np.arange(1979,2022+1)
    
    filelist = [f"/scratch/v46/gt3409/TMP_{turbulence_index}/TMP_{turbulence_index}_AUS-15_{source_id}_{experiment_id}_{member_id}_BOM_BARPA-R_v1-r1_6hr_{year}.nc" 
     for year in time_range]
    
    def _preprocess(ds):
        return (ds.sel(lat=mid_lat_slice)>p99).mean(["time", "lat", "lon"])
    
    ds = xr.open_mfdataset(filelist, use_cftime=True, preprocess=_preprocess, combine="nested", 
                           concat_dim="time").assign_coords({"run":name})
    ds["time"] = time_range
    # ds = ds.compute()
    delayed_results[name] = ds

evaluation_BARRA-R_r1i1p1f1
evaluation_ERA5_r1i1p1f1
CPU times: user 2.19 s, sys: 778 ms, total: 2.96 s
Wall time: 6.11 s


In [26]:
%%time
turbulence_index = "windspeed"
for run in i_historical:
    experiment_id, source_id, member_id = run.split("_")
    name = f"{experiment_id}_{source_id}_{member_id}"
    print(name)
    time_range = np.arange(1979,2014+1)
    
    filelist = [f"/scratch/v46/gt3409/TMP_{turbulence_index}/TMP_{turbulence_index}_AUS-15_{source_id}_{experiment_id}_{member_id}_BOM_BARPA-R_v1-r1_6hr_{year}.nc" 
     for year in time_range]
    
    def _preprocess(ds):
        return (ds.sel(lat=mid_lat_slice)>p99).mean(["time", "lat", "lon"])
    
    ds = xr.open_mfdataset(filelist, use_cftime=True,
                           preprocess=_preprocess, combine="nested", 
                           concat_dim="time").assign_coords({"run":name})
    ds["time"] = time_range
    # ds = ds.compute()
    delayed_results[name] = ds


historical_ACCESS-CM2_r4i1p1f1
historical_ACCESS-ESM1-5_r6i1p1f1
historical_CESM2_r11i1p1f1
historical_CMCC-ESM2_r1i1p1f1
historical_EC-Earth3_r1i1p1f1
historical_MPI-ESM1-2-HR_r1i1p1f1
historical_NorESM2-MM_r1i1p1f1
CPU times: user 13.1 s, sys: 5.39 s, total: 18.5 s
Wall time: 47.7 s


In [27]:
%%time
turbulence_index = "windspeed"
for run in i_future[1:]:
    experiment_id, source_id, member_id = run.split("_")
    name = f"{experiment_id}_{source_id}_{member_id}"
    print(name)
    time_range = np.arange(2015,2100+1)
    
    filelist = [f"/scratch/v46/gt3409/TMP_{turbulence_index}/TMP_{turbulence_index}_AUS-15_{source_id}_{experiment_id}_{member_id}_BOM_BARPA-R_v1-r1_6hr_{year}.nc" 
     for year in time_range]
    
    def _preprocess(ds):
        return (ds.sel(lat=mid_lat_slice)>p99).mean(["time", "lat", "lon"])
    
    ds = xr.open_mfdataset(filelist, use_cftime=True, preprocess=_preprocess, 
                           combine="nested", concat_dim="time").assign_coords({"run":name})
    ds["time"] = time_range
    # ds = ds.compute()
    delayed_results[name] = ds


ssp126_ACCESS-ESM1-5_r6i1p1f1
ssp126_CESM2_r11i1p1f1
ssp126_CMCC-ESM2_r1i1p1f1
ssp126_EC-Earth3_r1i1p1f1
ssp126_MPI-ESM1-2-HR_r1i1p1f1
ssp126_NorESM2-MM_r1i1p1f1
ssp370_ACCESS-CM2_r4i1p1f1
ssp370_ACCESS-ESM1-5_r6i1p1f1
ssp370_CESM2_r11i1p1f1
ssp370_CMCC-ESM2_r1i1p1f1
ssp370_EC-Earth3_r1i1p1f1
ssp370_MPI-ESM1-2-HR_r1i1p1f1
ssp370_NorESM2-MM_r1i1p1f1
ssp585_ACCESS-CM2_r4i1p1f1
ssp585_EC-Earth3_r1i1p1f1
CPU times: user 1min 6s, sys: 28.4 s, total: 1min 34s
Wall time: 4min


In [28]:
%%time
results = []
for run in i_evaluation:
     experiment_id, source_id, member_id = run.split("_")
     name = f"{experiment_id}_{source_id}_{member_id}"
     print(name)
     results.append(delayed_results[name].compute())

evaluation_ERA5_r1i1p1f1


2025-09-25 16:54:42,389 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('open_dataset-windspeed-getitem-0627b2b503bf250bbc2008490209f132', 3, 2, 6))" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2025-09-25 16:54:42,389 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('open_dataset-windspeed-getitem-0627b2b503bf250bbc2008490209f132', 2, 2, 6))" coro=<Worker.execute() done, defined at /g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker_state_machine.py:3615>> ended with CancelledError
2025-09-25 16:54:42,390 - distributed.worker.state_machine - WARNING - Async instruction for <Task cancelled name="execute(('open_dataset-windspeed-getitem-7285aa70e7b84b36bba6919f91fe7b50', 4, 2, 3))" coro=<

In [29]:
%%time
for run in i_historical:
     experiment_id, source_id, member_id = run.split("_")
     name = f"{experiment_id}_{source_id}_{member_id}"
     print(name)
     results.append(delayed_results[name].compute())

alysis3-24.04/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once
    handle._run()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/asyncio/events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/nanny.py", line 965, in run
    async with worker:
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/core.py", line 541, in __aexit__
    await self.close()
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/utils.py", line 837, in wrapper
    return await func(*args, **kwargs)
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/site-packages/distributed/worker.py", line 1644, in close
    await asyncio.to_thread(
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10

historical_ACCESS-CM2_r4i1p1f1


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/multiprocessing/__init__.py", line 16, in <module>

KeyboardInterrupt



In [30]:
ds_results = xr.concat(results, dim="run")
ds_results

    from . import context
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/multiprocessing/context.py", line 6, in <module>
    from . import reduction
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/multiprocessing/reduction.py", line 16, in <module>
    import socket
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/socket.py", line 54, in <module>
    import os, sys, io, selectors
  File "/g/data/hh5/public/apps/miniconda3/envs/analysis3-24.04/lib/python3.10/selectors.py", line 10, in <module>
    from collections.abc import Mapping
  File "<frozen importlib._bootstrap>", line 1027, in _find_and_load
  File "<frozen importlib._bootstrap>", line 1006, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 688, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 879, in exec_module
  File "<frozen importlib._bootstrap_external>", line 975, in get_code
  File "

ValueError: must supply at least one object to concatenate

In [ ]:
xr.plot.line(ds_results["windspeed"], x="time", hue="run", )

In [ ]:
%%time
for run in i_future[1:]:
     experiment_id, source_id, member_id = run.split("_")
     name = f"{experiment_id}_{source_id}_{member_id}"
     print(name)
     results.append(delayed_results[name].compute())

In [ ]:
ds_results = xr.concat(results, dim="run")
ds_results

In [ ]:
ds_results = xr.open_dataset("/scratch/v46/gt3409/timeseries_windspeed_mog_p99_ann.nc")


In [ ]:
list_evaluation = ['evaluation_ERA5_r1i1p1f1',
                   'evaluation_BARRA-R_r1i1p1f1',]

list_historical = ['historical_ACCESS-CM2_r4i1p1f1', 
                   'historical_ACCESS-ESM1-5_r6i1p1f1',
                   'historical_CESM2_r11i1p1f1', 
                   'historical_CMCC-ESM2_r1i1p1f1',
                   'historical_EC-Earth3_r1i1p1f1',
                   'historical_MPI-ESM1-2-HR_r1i1p1f1',
                   'historical_NorESM2-MM_r1i1p1f1',
                  ]

list_ssp126 = [
                 # 'ssp126_ACCESS-CM2_r4i1p1f1',
                 'ssp126_ACCESS-ESM1-5_r6i1p1f1',
                 'ssp126_CESM2_r11i1p1f1',
                 'ssp126_CMCC-ESM2_r1i1p1f1',
                 'ssp126_EC-Earth3_r1i1p1f1',
                 'ssp126_MPI-ESM1-2-HR_r1i1p1f1',
                 'ssp126_NorESM2-MM_r1i1p1f1',
              ]

list_ssp370 = ['ssp370_ACCESS-CM2_r4i1p1f1',
                 'ssp370_ACCESS-ESM1-5_r6i1p1f1',
                 'ssp370_CESM2_r11i1p1f1',
                 'ssp370_CMCC-ESM2_r1i1p1f1',
                 'ssp370_EC-Earth3_r1i1p1f1',
                 'ssp370_MPI-ESM1-2-HR_r1i1p1f1',
                 'ssp370_NorESM2-MM_r1i1p1f1',
              ]

list_ssp585 = ['ssp585_ACCESS-CM2_r4i1p1f1',
                 'ssp585_EC-Earth3_r1i1p1f1']

In [ ]:
ds_results.sel(run = list_evaluation + 
               list_historical + 
               list_ssp126 + 
               list_ssp370 +
               list_ssp585)

In [ ]:
ssp_colors = {"evaluation":"k",
              "historical":"grey",
              "ssp585":(149/255,27/255,30/255),
              "ssp370":(231/255,29/255,37/255),
              "ssp245":(247/255,148/255,32/255),
              "ssp126":(23/255,60/255,102/255),
              "ssp119":(0/255,173/255,207/255),}


In [ ]:
df = ds_results.sel(run = list_evaluation + 
               list_historical + 
               list_ssp126 + 
               list_ssp370 +
               list_ssp585).to_dataframe()["windspeed"].reset_index()
df["experiment"] = [x.split("_")[0] for x in df["run"]]


In [ ]:
window_size = 5
df["rolling_5y"] = df["windspeed"].rolling(window=window_size,).mean()
df

In [ ]:
sns.lineplot(df, x="time", y="windspeed", hue="experiment", errorbar=('ci', 100), palette=ssp_colors)


In [ ]:
sns.lineplot(df, x="time", y="rolling_5y", hue="experiment", errorbar=('ci', 100), palette=ssp_colors)

In [ ]:
list_ssp126 = ['ssp126_ACCESS-CM2_r4i1p1f1',
                 'ssp126_ACCESS-ESM1-5_r6i1p1f1',
                 'ssp126_CESM2_r11i1p1f1',
                 'ssp126_CMCC-ESM2_r1i1p1f1',
                 'ssp126_EC-Earth3_r1i1p1f1',
                 'ssp126_MPI-ESM1-2-HR_r1i1p1f1',
                 'ssp126_NorESM2-MM_r1i1p1f1',]

list_ssp370 = ['ssp370_ACCESS-CM2_r4i1p1f1',
                 'ssp370_ACCESS-ESM1-5_r6i1p1f1',
                 'ssp370_CESM2_r11i1p1f1',
                 'ssp370_CMCC-ESM2_r1i1p1f1',
                 'ssp370_EC-Earth3_r1i1p1f1',
                 'ssp370_MPI-ESM1-2-HR_r1i1p1f1',
                 'ssp370_NorESM2-MM_r1i1p1f1',]

list_ssp585 = ['ssp585_ACCESS-CM2_r4i1p1f1',
                 'ssp585_EC-Earth3_r1i1p1f1']

In [ ]:
xr.plot.line(ds_results.sel({"run": list(i_evaluation)})["windspeed"], x="time", hue="run", figsize=(12,12), color="k" );
xr.plot.line(ds_results.sel({"run": list(i_historical)})["windspeed"], x="time", hue="run", color="grey",  );
xr.plot.line(ds_results.sel({"run": list_ssp126[1:]})["windspeed"], x="time", hue="run", color="blue",  );
xr.plot.line(ds_results.sel({"run": list_ssp370})["windspeed"], x="time", hue="run", color="orange",  );
xr.plot.line(ds_results.sel({"run": list_ssp585})["windspeed"], x="time", hue="run", color="r",  );

In [ ]:
/

In [ ]:
%%time
results = []
for run in i_evaluation:
     experiment_id, source_id, member_id = run.split("_")
     name = f"{experiment_id}_{source_id}_{member_id}"
     print(name)
     results.append(delayed_results[name].compute())

In [ ]:
%%time
for run in i_historical:
     experiment_id, source_id, member_id = run.split("_")
     name = f"{experiment_id}_{source_id}_{member_id}"
     print(name)
     results.append(delayed_results[name].compute())

In [ ]:
ds_results = xr.concat(results, dim="run")
ds_results

In [ ]:
xr.plot.line(ds_results["windspeed"], x="time", hue="run", )

In [ ]:
%%time
for run in i_future[1:]:
     experiment_id, source_id, member_id = run.split("_")
     name = f"{experiment_id}_{source_id}_{member_id}"
     print(name)
     results.append(delayed_results[name].compute())

In [ ]:
ds_results = xr.concat(results, dim="run")
ds_results

In [ ]:
ds_results.to_netcdf("/scratch/v46/gt3409/timeseries_windspeed_p99_ann.nc")

In [ ]:
list_ssp126 = ['ssp126_ACCESS-CM2_r4i1p1f1',
                 'ssp126_ACCESS-ESM1-5_r6i1p1f1',
                 'ssp126_CESM2_r11i1p1f1',
                 'ssp126_CMCC-ESM2_r1i1p1f1',
                 'ssp126_EC-Earth3_r1i1p1f1',
                 'ssp126_MPI-ESM1-2-HR_r1i1p1f1',
                 'ssp126_NorESM2-MM_r1i1p1f1',]

list_ssp370 = ['ssp370_ACCESS-CM2_r4i1p1f1',
                 'ssp370_ACCESS-ESM1-5_r6i1p1f1',
                 'ssp370_CESM2_r11i1p1f1',
                 'ssp370_CMCC-ESM2_r1i1p1f1',
                 'ssp370_EC-Earth3_r1i1p1f1',
                 'ssp370_MPI-ESM1-2-HR_r1i1p1f1',
                 'ssp370_NorESM2-MM_r1i1p1f1',]

list_ssp585 = ['ssp585_ACCESS-CM2_r4i1p1f1',
                 'ssp585_EC-Earth3_r1i1p1f1']

In [ ]:
xr.plot.line(ds_results.sel({"run": list(i_evaluation)})["windspeed"], x="time", hue="run", figsize=(12,12), color="k" );
xr.plot.line(ds_results.sel({"run": list(i_historical)})["windspeed"], x="time", hue="run", color="grey",  );
xr.plot.line(ds_results.sel({"run": list_ssp126[1:]})["windspeed"], x="time", hue="run", color="blue",  );
xr.plot.line(ds_results.sel({"run": list_ssp370})["windspeed"], x="time", hue="run", color="orange",  );
xr.plot.line(ds_results.sel({"run": list_ssp585})["windspeed"], x="time", hue="run", color="r",  );